# Import

In [ ]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
import utils as u

from torchinfo import summary
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from transformers import SegformerForSemanticSegmentation

import warnings
warnings.filterwarnings("ignore")
print(torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

# Hyperparameters

In [ ]:
DATA_PATH = 'data'
MODEL_PATH = 'model'
MODEL_CHECKPOINT = "nvidia/mit-b2"
NAME = f'Segformer'

IMG_SIZE = 256
N_CHANNELS = 3
BATCH_SIZE = 16

SEED = 5
K_FOLDS = 5
TRAIN_SPLIT = 0.95
DATA_LIMIT = None

LR = 1e-4
LR_ADJ = 'None'
EPOCHS = 100

PATIENCE = 15
DELTA = 0.01

hyperparams = {
    "MODEL_CHECKPOINT": MODEL_CHECKPOINT,
    "NAME": NAME,
    "IMG_SIZE": IMG_SIZE,
    "BATCH_SIZE": BATCH_SIZE,
    "SEED": SEED,
    "TRAIN_SPLIT": TRAIN_SPLIT,
    "DATA_LIMIT": DATA_LIMIT,
    "LR": LR,
    "LR_ADJ": LR_ADJ,
    "EPOCHS": EPOCHS,
    "PATIENCE": PATIENCE,
    "DELTA": DELTA
}

json_path = f"{MODEL_PATH}/{NAME}/hyperparameters.json"
os.makedirs(f"{MODEL_PATH}/{NAME}", exist_ok=True)
with open(json_path, "w") as f:
    json.dump(hyperparams, f, indent=4)

# Data Setup

In [ ]:
dataset = u.get_image_path(f'{DATA_PATH}/train_val', limit=DATA_LIMIT, seed=SEED)
train_dataset, val_dataset = train_test_split(dataset, train_size=TRAIN_SPLIT, random_state=SEED, shuffle=True)

train_loader= DataLoader(u.LandsatDataset(dataset=train_dataset, size=IMG_SIZE, do_augmentation=True), shuffle=True, batch_size=BATCH_SIZE, drop_last=True, num_workers=8)
val_loader= DataLoader(u.LandsatDataset(dataset=val_dataset, size=IMG_SIZE, do_augmentation=False), shuffle=False, batch_size=BATCH_SIZE, drop_last=True, num_workers=8)

print(f"Dataset size: \t{len(dataset)}")
print(f"Training: \t#{len(train_dataset)} (samples)\t#{len(train_loader)} (batchs)")
print(f"Validation: \t#{len(val_dataset)} (samples)\t#{len(val_loader)} (batchs)\n")

In [ ]:
# dataset = u.get_image_path(f'{DATA_PATH}/train_val', limit=DATA_LIMIT, seed=SEED)
# kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=SEED)
# for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
#     print(f"Fold {fold+1}/{K_FOLDS}")

#     train_dataset = [dataset[i] for i in train_idx]
#     val_dataset = [dataset[i] for i in val_idx]
#     break

# train_loader= DataLoader(u.LandsatDataset(dataset=train_dataset, size=IMG_SIZE, do_augmentation=True), shuffle=True, batch_size=BATCH_SIZE, drop_last=True, num_workers=4, prefetch_factor=8)
# val_loader= DataLoader(u.LandsatDataset(dataset=val_dataset, size=IMG_SIZE, do_augmentation=False), shuffle=False, batch_size=BATCH_SIZE, drop_last=True, num_workers=4, prefetch_factor=8)

# print(f"Dataset size: \t{len(dataset)}")
# print(f"Training: \t#{len(train_dataset)} (samples)\t#{len(train_loader)} (batchs)")
# print(f"Validation: \t#{len(val_dataset)} (samples)\t#{len(val_loader)} (batchs)\n")

In [ ]:
# u.plotter(train_loader)
# u.plotter(val_loader)

# Model Setup

In [ ]:
id2label = {0: 'background', 1: 'fire'}
label2id = {label: id for id, label in id2label.items()}
num_labels = len(id2label)

model = SegformerForSemanticSegmentation.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)
model.config.image_size = IMG_SIZE

for param in model.parameters():
    param.requires_grad = False

for param in model.decode_head.parameters():
    param.requires_grad = True
model.decode_head.classifier = torch.nn.Conv2d(768, num_labels, kernel_size=(1, 1))

summary(model=model, 
        input_size=(BATCH_SIZE, N_CHANNELS, IMG_SIZE, IMG_SIZE), # (batch_size, color_channels, height, width)
        col_names=["input_size", "output_size", "trainable"],
        row_settings=["var_names"]
)

# Training Setup

In [ ]:
loss = nn.CrossEntropyLoss() # to be changed 
optimizer = optim.AdamW(model.parameters(), lr=LR) # lr to be changed 
u.seed_everything(SEED)
u.train_and_validate(
    name=NAME,
    model=model.cuda(),
    train_=train_loader,
    val_=val_loader,
    loss_=loss,
    optimizer=optimizer,
    scheduler_type=LR_ADJ,
    epochs=EPOCHS,
    delta=DELTA,
    early_stop=PATIENCE,
    output_dir=MODEL_PATH
)

# Testing Setup

In [ ]:
test_dataset = u.get_image_path(f'{DATA_PATH}/test', seed=SEED)
test_loader= DataLoader(u.LandsatDataset(dataset=test_dataset, size=IMG_SIZE, do_augmentation=False), shuffle=False, batch_size=10)
print(f"Test: #{len(test_dataset)} (samples)\t#{len(test_loader)} (batchs)")

In [ ]:
model = SegformerForSemanticSegmentation.from_pretrained(
    f'D:\DATA-5000\model\{NAME}'
).cuda()
img, gt, pr, path = u.test(NAME, model, test_loader, MODEL_PATH, th=0.5)

In [ ]:
u.plotter_results(images=img, masks=gt, predictions=pr, path=path, num_samples= 10)

In [ ]:
u.plotter_history(f'model\{NAME}\history.json')